In [1]:
import random
import sys

import nltk
import itertools
from collections import defaultdict
from nltk.tokenize import word_tokenize

import numpy as np

import pickle
import pprint
import emoji
import re
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers , activations , models , preprocessing , utils



Using TensorFlow backend.


In [2]:
import numpy as np
import tensorflow as tf
import keras
import pickle
from pprint import pprint
from pickle import load
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
print( tf.VERSION )
from tensorflow.keras import layers , activations , models , preprocessing



1.13.1


In [3]:
WHITELIST = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ' # space is included in whitelist
BLACKLIST = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\''
FILENAME = 'tousend_mes.txt'
limit = {
        'maxq' : 20,
        'minq' : 1,
        'maxa' : 22,
        'mina' : 1
        }

UNK = 'unk'
# VOCAB_SIZE = 6000

In [4]:
'''
 read lines from file
     return [list of lines]
'''
def read_lines(filename):
    return open(filename, encoding='UTF-8').read().split('\n')

In [5]:
lines = read_lines(FILENAME)
len(lines)


1050

In [6]:
# функции для чистки
'''
 split sentences in one line
  into multiple lines
    return [list of lines]
'''
def split_line_ANS(line):
    split_line = re.split('\.|\!|\?', line)
    for el in split_line:
        if re.search('[0-9a-zA-Z]', el):
            return el # только первую содержательную часть ответа возвращает, до знака препинания
    else:
        return 'EMPTY'
    
def split_line_QUES(line):
    split_line = re.split('\.|\!|\?', line)
    for el in split_line[-1::-1]:
        if re.search('[0-9a-zA-Z]', el):
            return el # только последнюю содержательную часть ответа возвращает, до знака препинания
    else:
        return 'EMPTY'    

def clean_line(lines, whitelist):
    
    clean_lines = []
    for i in range(0, len(lines)):
        line = lines[i]
        clean_line = (''.join(i for i in line if i not in emoji.UNICODE_EMOJI ))
        clean_line = re.sub('<@\d+>', '', clean_line)
        clean_line = re.sub('<:.+>', '', clean_line)
        if i % 2 == 0:
            clean_line = split_line_QUES(clean_line)
        else:
            clean_line = split_line_ANS(clean_line)
            

        
        clean_line = ''.join([ ch for ch in clean_line if ch in whitelist ])
        clean_line = clean_line.lower()
        clean_lines.append(clean_line)
    return clean_lines


In [7]:
lines = clean_line(lines, WHITELIST)

In [8]:
len(lines)

1050

In [9]:
with open("cleaned_mes.txt", 'w') as f:
    for line in lines:
        f.write(line + '\n')


In [10]:
filtered_q, filtered_a = [], []
for i in range(0, len(lines), 2): # перешагиваем через 1
    qlen, alen = len(lines[i].split(' ')), len(lines[i+1].split(' '))
    if qlen >= limit['minq'] and qlen <= limit['maxq']: # если и вопрос и ответ укладываются в длину
        if alen >= limit['mina'] and alen <= 20:
            filtered_q.append(lines[i]) #то  добавляем их в датасет. Если нет, то просто игнорим.
            filtered_a.append(lines[i+1])


In [11]:
len(filtered_q), len(filtered_a)

(487, 487)

In [12]:
with open("QUESTIONS_mes.txt", 'w') as f:
    for line in filtered_q:
        f.write(line + '\n')

In [13]:
with open("ANSWERS_mes.txt", 'w') as f:
    for line in filtered_a:
        f.write(line + '\n')

In [14]:
filtered_a_TAGS = list()
for line in filtered_a:
    filtered_a_TAGS.append( '<START> ' + line + ' <END>' )  

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts( filtered_q+filtered_a_TAGS ) 
tokenized_questions = tokenizer.texts_to_sequences( filtered_q ) 


In [16]:
padded_questions = pad_sequences( tokenized_questions , maxlen=limit['maxq'] , padding='post' )
encoder_input_data = np.array( padded_questions )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))


Encoder input data shape -> (487, 20)


In [17]:
word_dict = tokenizer.word_index
num_tokens = len( word_dict )+1
print( 'Number of tokens = {}'.format( num_tokens))


Number of tokens = 1113


In [22]:
tokenized_answers = tokenizer.texts_to_sequences( filtered_a_TAGS ) 
padded_answers = pad_sequences( tokenized_answers , maxlen=limit['maxa'], padding='post' )
decoder_input_data = np.array( padded_answers )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))




Decoder input data shape -> (487, 22)


In [21]:
len(np.unique(padded_answers))

677

In [23]:
padded_answers.shape

(487, 22)

 #### decoder_target_data

In [24]:
decoder_target_data = list()
for token_seq in tokenized_answers:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
# padded_answers = pad_sequences( decoder_target_data , maxlen=limit['maxa'], padding='post' )
# onehot_answers = utils.to_categorical( padded_answers , num_tokens )
# decoder_target_data = np.array( onehot_answers )
# print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

In [26]:
decoder_target_data[0:4]

[[23, 18, 49, 876, 7, 10, 877, 8, 878, 2],
 [13, 36, 12, 43, 370, 76, 2],
 [75, 21, 2],
 [75, 2]]

In [27]:
len(decoder_target_data)

487

In [31]:
padded_answers = pad_sequences( decoder_target_data , maxlen=limit['maxa'], padding='post' )


In [50]:
padded_answers

array([[ 23,  18,  49, ...,   0,   0,   0],
       [ 13,  36,  12, ...,   0,   0,   0],
       [ 75,  21,   2, ...,   0,   0,   0],
       ...,
       [  3, 347, 162, ...,   0,   0,   0],
       [ 29,   2,   0, ...,   0,   0,   0],
       [  3, 270,  24, ...,   0,   0,   0]])

In [51]:
print(len(padded_answers))
print(padded_answers.shape)

487
(487, 22)


In [32]:
type(padded_answers)

numpy.ndarray

In [37]:
print(len(np.unique(padded_answers)))
np.unique(padded_answers)[0:200]


677


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 144,
       145, 147, 148, 151, 152, 153, 154, 155, 156, 158, 159, 160, 162,
       163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175,
       177, 178, 179, 183, 185, 186, 187, 188, 189, 190, 191, 19

In [40]:
num_tokens

1113

In [41]:
22*487*1113

11924682

In [42]:
onehot_answers = utils.to_categorical( padded_answers , num_tokens )

In [43]:
print(type(onehot_answers))
print(onehot_answers.shape)

<class 'numpy.ndarray'>
(487, 22, 1113)


In [48]:
print(len(onehot_answers[0]))
print((onehot_answers[0]))
print(len(onehot_answers[0][0]))
print((onehot_answers[0][0]))
                            

22
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
1113
[0. 0. 0. ... 0. 0. 0.]


In [156]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_tokens, 200 , mask_zero=True ) (encoder_inputs)

encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_tokens, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )

model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    222600      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    222600      input_2[0][0]                    
_____________________________________

In [157]:
filepath="weights-CHAT-discord_improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [160]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=50, epochs=20, callbacks=callbacks_list )

Epoch 1/20
450/487 [==========================>...] - ETA: 0s - loss: 0.9604
Epoch 00001: loss improved from 1.00936 to 0.96458, saving model to weights-CHAT-discord_improvement-01-0.9646.hdf5
487/487 [==============================] - 3s 6ms/sample - loss: 0.9646
Epoch 2/20
450/487 [==========================>...] - ETA: 0s - loss: 0.9196
Epoch 00002: loss improved from 0.96458 to 0.91790, saving model to weights-CHAT-discord_improvement-02-0.9179.hdf5
487/487 [==============================] - 3s 6ms/sample - loss: 0.9179
Epoch 3/20
450/487 [==========================>...] - ETA: 0s - loss: 0.8889
Epoch 00003: loss improved from 0.91790 to 0.88396, saving model to weights-CHAT-discord_improvement-03-0.8840.hdf5
487/487 [==============================] - 3s 6ms/sample - loss: 0.8840
Epoch 4/20
450/487 [==========================>...] - ETA: 0s - loss: 0.8295
Epoch 00004: loss improved from 0.88396 to 0.84436, saving model to weights-CHAT-discord_improvement-04-0.8444.hdf5
487/487 [===

In [161]:
max_question_len = encoder_input_data.shape[1]
max_answer_len = decoder_input_data.shape[1]

In [162]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [163]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        try:
            tokens_list.append( word_dict[ word ] ) 
        except:
            continue
    return pad_sequences( [tokens_list] , maxlen=max_question_len , padding='post')

In [164]:
enc_model , dec_model = make_inference_models()

for _ in range(10): # 10 вопросов можно
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )#1.   First, we take a question as input and predict the state values using `enc_model`.

    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )#2.   We set the state values in the decoder's LSTM.

        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_answer_len:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question : how are you
 im good how about you end
Enter question : whats up
 hajime hinata tiki from fe chiaki nanami and joshua from fe end
Enter question : up
 wassup end
Enter question : whats
 wassup end
Enter question : whats up
 hajime hinata tiki from fe chiaki nanami and joshua from fe end
Enter question : how about mango
 nice end
Enter question : nice? he is evil
 hajime hinata tiki from fe chiaki nanami and joshua from fe end
Enter question : olala olala be
 hahaha end
Enter question : you are a nice chatbot
 nice i forgot how haha end
Enter question : very informative
 wassup end
